In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import re
import requests
import time

import swagger_client
from swagger_client.rest import ApiException

In [2]:
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [3]:
load_dotenv()
AEMET_KEY = os.getenv("KEY_AEMET")
configuration = swagger_client.Configuration()
configuration.api_key['api_key'] = AEMET_KEY
api_instance = swagger_client.ValoresClimatologicosApi(swagger_client.ApiClient(configuration))

In [4]:
df15 = pd.read_csv('data/Temp15-16_meteo.csv')
df16 = pd.read_csv('data/Temp16-17_meteo.csv')
df17 = pd.read_csv('data/Temp17-18_meteo.csv')

In [76]:
df16.loc[(df16.rachaviento == '-')]

,Equipo Local,Goles,Ataque,Medio,Defensa,Goles_Acumulados,Equipo Visitante,Goles.1,Ataque.1,Medio.1,...,Resultado,cod_est,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento
323,Osasuna,2,75,73,73,32,Sporting de Gijón,2,75,73,...,1,9263D,1.9,12.6,23.3,0.0,965.9,973.0,0.8,-


In [77]:
est_meteo_mod16 = {

'Osasuna': {'prov':'Navarra',
            'cod_est':'9294E'},

}

In [78]:
df16.loc[(df16.rachaviento == '-'), 'cod_est'] = df16.loc[(df16.rachaviento == '-')]['Equipo Local'].apply(lambda x: est_meteo_mod16[x]['cod_est'])

In [79]:
for index, row in df16.loc[(df16.rachaviento == '-')].iterrows():
    print(f'Obteniendo datos de la fila: {index}')
    fecha_ini = row.Día + 'T00:00:00UTC'
    fecha_fin = row.Día + 'T23:59:59UTC'
    estacion = row.cod_est
    
    try:
        respuesta_api = api_instance.climatologas_diarias_(fecha_ini, fecha_fin, estacion)
        url_respuesta = respuesta_api.datos
        response = requests.request("GET", url_respuesta)
        print(response)
        print('\n\n')
        meteo = json.loads(response.text)[0]
        
        try:
            df16.loc[index, 'rachaviento'] = meteo['racha']
        except:
            df16.loc[index, 'rachaviento'] = '-'

        time.sleep(5)
    except ApiException as e:
        print ('Algo ha fallado')
    

Obteniendo datos de la fila: 323
<Response [200]>





In [80]:
df16.iloc[323]

Equipo Local                    Osasuna
Goles                                 2
Ataque                               75
Medio                                73
Defensa                              73
Goles_Acumulados                     32
Equipo Visitante      Sporting de Gijón
Goles.1                               2
Ataque.1                             75
Medio.1                              73
Defensa.1                            73
Goles_Acumulados.1                   34
Día                          2017-04-22
Hora                              18:30
Jornada                              33
Resultado                             1
cod_est                           9294E
tmin                                1.9
tmed                               12.6
tmax                               23.3
prec                                0.0
presmin                           965.9
presmax                           973.0
velmedia                            0.8
rachaviento                         7,5


In [35]:
def change_prec(val):
    if val == 'Ip':
        return '0,05'
    elif val == '-':
        return '0,0'
    else:
        return val
    
df16.prec = df16.prec.apply(lambda x: change_prec(x))

In [83]:
df16.to_csv('data/Temp16-17_meteo.csv', index=False)

In [73]:
df16.loc[:,'tmin':].head()

,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento
0,26.9,32.8,38.6,0.0,1002.8,1006.0,3.3,8.3
1,18.1,21.0,23.8,0.05,1001.6,1011.0,5.0,12.8
2,23.5,26.4,29.2,0.0,1011.6,1019.0,7.2,13.9
3,16.1,26.1,36.1,0.0,949.6,952.9,2.8,10.8
4,21.6,29.7,37.8,0.0,1009.8,1013.4,4.2,13.3


In [81]:
df16.loc[:,'tmin':] = df16.loc[:,'tmin':].apply(lambda x: x.str.replace(',', '.'))
df16.loc[:,'tmin':] = df16.loc[:,'tmin':].apply(pd.to_numeric, downcast='float')

In [82]:
df16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Equipo Local        380 non-null    object 
 1   Goles               380 non-null    int64  
 2   Ataque              380 non-null    int64  
 3   Medio               380 non-null    int64  
 4   Defensa             380 non-null    int64  
 5   Goles_Acumulados    380 non-null    int64  
 6   Equipo Visitante    380 non-null    object 
 7   Goles.1             380 non-null    int64  
 8   Ataque.1            380 non-null    int64  
 9   Medio.1             380 non-null    int64  
 10  Defensa.1           380 non-null    int64  
 11  Goles_Acumulados.1  380 non-null    int64  
 12  Día                 380 non-null    object 
 13  Hora                380 non-null    object 
 14  Jornada             380 non-null    int64  
 15  Resultado           380 non-null    int64  
 16  cod_est 

In [ ]:
est_meteo = {
'Eibar': {'prov':'Gipuzkoa',
         'cod_est':'1050J'},
'Granada': {'prov':'Granada',
            'cod_est':'5530E'},
'Sporting de Gijón': {'prov':'Asturias',
                      'cod_est':'1207U'},
'Girona': {'prov':'Girona',
           'cod_est':'0367'},
'Espanyol': {'prov':'Barcelona',
             'cod_est':'0076'},
'Rayo Vallecano': {'prov':'Madrid',
                   'cod_est':'3195'},
'Athletic Club': {'prov':'Bizkaia',
                  'cod_est':'1082'},
'Alavés': {'prov':'Araba',
           'cod_est':'9091R'},
'Osasuna': {'prov':'Navarra',
            'cod_est':'9263D'},
'Deportivo de La Coruña': {'prov':'A Coruña',
                           'cod_est':'1387'},
'Sevilla': {'prov':'Sevilla',
            'cod_est':'5783'},
'Real Madrid': {'prov':'Madrid',
                'cod_est':'3195'},
'Leganés': {'prov':'Madrid',
            'cod_est':'3195'},
'Levante': {'prov':'Valencia',
            'cod_est':'8414A'},
'Barcelona': {'prov':'Barcelona',
              'cod_est':'0076'},
'Getafe': {'prov':'Madrid',
           'cod_est':'3195'},
'Málaga': {'prov':'Málaga',
           'cod_est':'6156X'},
'Villarreal': {'prov':'Castellón',
               'cod_est':'8500A'},
'Atlético de Madrid': {'prov':'Madrid',
                       'cod_est':'3195'},
'Real Sociedad': {'prov':'Gipuzkoa',
                  'cod_est':'1014A'},
'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C659M'},
'Celta de Vigo': {'prov':'Pontevedra',
         'cod_est':'1495'},
'Betis': {'prov':'Sevilla',
          'cod_est':'5783'},
'Valencia': {'prov':'Valencia',
             'cod_est':'8414A'}

}
